In [ ]:
'''
- Preparing Grayscale Data for Pretrained Models (e.g., VGG16, InceptionV3)

Most pretrained models (like VGG16, ResNet, or Inception) were trained on ImageNet, which contains RGB images of size 224×224.
So, if we’re working with grayscale datasets (like MNIST having 32x32), we need to:
1.Convert grayscale images → 3-channel RGB.
2.Resize 28×28 → 224×224 pixels.

'''
# -------------------------------------------
# Converting Grayscale Images to RGB & Resizing
# -------------------------------------------

# Step 1: Convert grayscale (28x28) → RGB (28x28x3)
# Repeat the single grayscale channel three times to simulate RGB.
X_train_rgb = np.repeat(X_train[..., np.newaxis], 3, axis=-1)
X_test_rgb = np.repeat(X_test[..., np.newaxis], 3, axis=-1)

# Step 2: Resize images to 224x224 (required by VGG16 / InceptionV3)
X_train_incep = tf.image.resize(X_train_rgb, (224, 224)).numpy()
X_test_incep = tf.image.resize(X_test_rgb, (224, 224)).numpy()

# Display the new shapes to confirm
print("X_train_rgb shape:", X_train_rgb.shape)
print("X_test_rgb shape:", X_test_rgb.shape)
print("X_train resized shape:", X_train_incep.shape)
print("X_test resized shape:", X_test_incep.shape)

'''
⚠️ Note: Since CIFAR-10 images are already RGB (32×32×3), we only need to resize them to (224×224×3) for pretrained models like VGG16 or InceptionV3.
'''


In [ ]:
# ================================================================
# 🧠 CIFAR-10 Classification using Pretrained VGG16 (Transfer Learning)
# ================================================================
'''
- CIFAR-10 has 60,000 small (32×32) images in 10 categories.
- VGG16 requires 224×224 images, so we resize them.
- preprocess_input() scales the data like ImageNet (the dataset VGG16 was trained on).
- The base layers of VGG16 are frozen so we don’t retrain them, only train the new Dense layers added on top.
- Transfer learning lets us use pretrained knowledge on large datasets (like ImageNet) for smaller ones (like CIFAR-10).
'''
# -----------------------------
# Import Required Libraries
# -----------------------------
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------
# Load and Explore CIFAR-10
# -----------------------------
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("Original CIFAR-10 shapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

# Show some random samples
plt.figure(figsize=(6,3))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(X_train[i])
    plt.title(f"Label: {y_train[i][0]}")
    plt.axis('off')
plt.suptitle("Sample CIFAR-10 Images", fontsize=14)
plt.show()

# -----------------------------
# Preprocess and Resize
# -----------------------------
# Resize from (32x32) to (224x224) for VGG16 input
X_train_resized = tf.image.resize(X_train, (224, 224))
X_test_resized = tf.image.resize(X_test, (224, 224))

# Convert to numpy arrays
X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)

# Apply VGG16 preprocessing (scales pixels to match ImageNet expectations)
X_train_pre = preprocess_input(X_train_resized)
X_test_pre = preprocess_input(X_test_resized)

# One-hot encode labels
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# -----------------------------
# Load Pretrained VGG16 Base
# -----------------------------
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze convolutional base
for layer in base_model.layers:
    layer.trainable = False

# -----------------------------
# Build Transfer Learning Model
# -----------------------------
vgg_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

vgg_model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

vgg_model.summary()

# -----------------------------
# Train the Model
# -----------------------------
history = vgg_model.fit(X_train_pre, y_train_cat,
                        epochs=5,
                        batch_size=64,
                        validation_split=0.2)

# -----------------------------
# Evaluate the Model
# -----------------------------
test_loss, test_acc = vgg_model.evaluate(X_test_pre, y_test_cat)
print(f"\n✅ VGG16 (Transfer Learning) Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
# ================================================================
# 🌐 CIFAR-10 Classification using Pretrained InceptionV3 (Transfer Learning)
# ================================================================

# -----------------------------
# Import Required Libraries
# -----------------------------
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Load and Explore CIFAR-10
# -----------------------------
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("CIFAR-10 shapes:")
print("X_train:", X_train.shape, "y_train:", y_train.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)

# Show a few sample images
plt.figure(figsize=(6,3))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(X_train[i])
    plt.title(f"Label: {y_train[i][0]}")
    plt.axis('off')
plt.suptitle("Sample CIFAR-10 Images", fontsize=14)
plt.show()

# -----------------------------
# Preprocess & Resize
# -----------------------------
# InceptionV3 expects 224x224 images
X_train_resized = tf.image.resize(X_train, (224, 224))
X_test_resized = tf.image.resize(X_test, (224, 224))

# Convert to numpy
X_train_resized = np.array(X_train_resized)
X_test_resized = np.array(X_test_resized)

# Preprocess input as required by InceptionV3
X_train_pre = preprocess_input(X_train_resized)
X_test_pre = preprocess_input(X_test_resized)

# -----------------------------
# Build Pretrained Model
# -----------------------------
# Load base InceptionV3 model
base_model_incep = InceptionV3(weights='imagenet', include_top=False, input_shape=(224,224,3))

# -----------------------------
# Explanation for New Learners
# -----------------------------
# - include_top=False removes the final classifier head (used for ImageNet)
# - We'll add our own Dense layers suited for 10 CIFAR classes
# - "Freezing" layers means their weights won't change during training
# - Later, we can unfreeze some for fine-tuning if needed

# Freeze base layers
for layer in base_model_incep.layers:
    layer.trainable = False

# -----------------------------
# Add Custom Classification Head
# -----------------------------
inception_model = Sequential([
    base_model_incep,
    GlobalAveragePooling2D(), # for flattened . we use max for 2d. gobal can be used for both flattened as well as 2d
    # Replaces Flatten for better feature aggregation
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

# Compile
inception_model.compile(optimizer=Adam(learning_rate=0.0001),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

inception_model.summary()

# -----------------------------
# Train the Model
# -----------------------------
history_incep = inception_model.fit(X_train_pre, y_train,
                                    validation_split=0.2,
                                    epochs=5,
                                    batch_size=64)

# -----------------------------
# Evaluate the Model
# -----------------------------
test_loss, test_acc = inception_model.evaluate(X_test_pre, y_test)
print(f"\n✅ InceptionV3 (Transfer Learning) Test Accuracy: {test_acc*100:.2f}%")


'''
# fine tuning : include_top
# top of any model dependent on dataset . so we remove the head because we dont need there head . we would be using our own head
# pre training vs fine tuning
# pretraining : using a architecture but training on our own . discarding its previous weights and learning
# fine tune :keep train weights as it is . remove their top/head . add our head . freeze the previous weights of original architectire. this way only weights of our own layers gets trained
# unfreeze some layers in fine tune if the previous original model learning a bit cotrdictory with outr dataset . it would upadte weights of both : our new head and unfreezed layers . better convergence

# transfer learning : Pretrain + finetune
'''


In [ ]:
# ================================================================
# 🌈 Data Augmentation and Training with InceptionV3
# ================================================================

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# -----------------------------
# Why Data Augmentation?
# -----------------------------
# Neural networks often overfit when training data is limited or repetitive.
# Data augmentation artificially increases dataset diversity by applying random
# transformations (rotations, shifts, flips) to input images during training.

# Define augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=15,        # Randomly rotate images by ±15 degrees
    width_shift_range=0.1,    # Shift images horizontally by ±10%
    height_shift_range=0.1,   # Shift images vertically by ±10%
    horizontal_flip=True      # Randomly flip images horizontally
)

# Fit the generator to the training data
# (required to compute internal statistics, though not needed for all transforms)
datagen.fit(X_train_pre)

# -----------------------------
# Train the Model with Augmentation
# -----------------------------
# The generator dynamically produces augmented images during training.
# This means the model never sees the exact same image twice — improving robustness.

history_incep = inception_model.fit(
    datagen.flow(X_train_pre, y_train, batch_size=64),  # use generator instead of raw data
    epochs=3,
    validation_data=(X_test_pre, y_test),               # evaluate on unaugmented test data
    verbose=1
)
'''
- ImageDataGenerator: Real-time augmentation that slightly alters images each epoch to simulate new data.

- fit() vs. flow():
fit() prepares the generator (not always needed).
flow() feeds batches of augmented data to the model.

- Goal: This improves model generalization and reduces overfitting, especially when fine-tuning pretrained networks.
'''